In [55]:
import numpy as np
import pandas as pd
import time

In [56]:
dataset = pd.read_csv('as-skitter.csv')
nodes = np.concatenate((dataset['node_1'].unique(), dataset['node_2'].unique()))
nodes = np.unique(nodes)
e = 0
n = nodes.size

In [57]:
"""
for j in range(0, len(nodes)):
    dataset = dataset.replace(nodes[j], j)
"""

'\nfor j in range(0, len(nodes)):\n    dataset = dataset.replace(nodes[j], j)\n'

In [58]:
edges = dataset.iloc[:, :].values

In [59]:
G = {}

for i in nodes:
    #print(i)
    G[i] = []
    
for ed in edges:
    if ed[0] != ed[1]:
        G[ed[0]].append(ed[1])
        G[ed[1]].append(ed[0])
        e += 1

In [60]:
def densest(G, nodes, n, e):
    prev_core = None
    densities = []
    deg = {}
    bucket = []
    pos = {}
    vert = []
    edges = e
    vertices = n
    me = e
    mv = n
    posmc = None
    mcore = 0
    mden = 0
    md = 0
    
    for v in nodes:
        deg[v] = 0
        vert.append(None)
        pos[v] = None
    
    #easily parallelizable
    for v in nodes:
        d = 0
        for u in G[v]:
            d += 1
        
        deg[v] = d
        if d > md:
            md = d
    
    for d in range(md + 1):
        bucket.append(0)
    
    for v in nodes:
        bucket[deg[v]] += 1
        
    start = 0
    for d in range(md + 1):
        num = bucket[d]
        bucket[d] = start
        start += num
    
    #easily parallalizable
    for v in nodes:
        pos[v] = bucket[deg[v]]
        vert[pos[v]] = v
        bucket[deg[v]] += 1
    
    #easily parallalizable
    for d in range(md, 0, -1):
        bucket[d] = bucket[d - 1]
    bucket[0] = 0
    
    for i in range(n):
        v = vert[i]
        core = deg[v]
        
        if prev_core != core:
            density = edges / (n - i)
            densities.append([core, density])
            if density > mden:
                mden = density
                mcore = core
                me = edges
                mv = n - i
            prev_core = core
        
        for u in G[v]:
            if pos[u] > pos[v]:
                edges -= 1
                
            if deg[u] > deg[v]:
                du = deg[u]
                pu = pos[u]
                pw = bucket[du]
                w = vert[pw]
                
                if u != w:
                    pos[u] = pw
                    vert[pu] = w
                    pos[w] = pu
                    vert[pw] = u
                    
                bucket[du] += 1
                deg[u] -= 1
    
    #print(mden, me, mv)
    posmc = bucket[mcore]
    for i in range(posmc - 1, -1, -1):
        v = vert[i]
        
        if deg[v] < mden:
            break
            
        legits = 0
        for u in G[v]:
            if pos[u] >= posmc:
                legits += 1
        
        #print(mden, deg[v], legits)
        if legits > mden:
            w = vert[posmc - 1]
            pos[v] = posmc - 1
            vert[i] = w
            pos[w] = i
            vert[posmc - 1] = v
            posmc -= 1
            me += legits
            mv += 1
            mden = me / mv
            
    return densities, vert, pos, posmc, me, mv, mden

In [61]:
start_time = time.clock()
densities, vert, pos, posmc, me, mv, mden = densest(G, nodes, n, e)
print(time.clock() - start_time, "seconds")

49.873841999999996 seconds


In [62]:
me, mv

(41482, 464)

In [63]:
densities

[[1, 6.540438512981788],
 [2, 7.420417753855822],
 [3, 8.812676117842583],
 [4, 10.164331988317496],
 [5, 11.559272854261435],
 [6, 12.974082177476248],
 [7, 14.402998509131569],
 [8, 15.829952714857535],
 [9, 17.27580124106905],
 [10, 18.765269765603698],
 [11, 20.162396999508637],
 [12, 21.609453749012776],
 [13, 23.019043327169843],
 [14, 24.408320517412097],
 [15, 25.803030753407963],
 [16, 27.11456756684989],
 [17, 28.43833307458582],
 [18, 29.66193899675584],
 [19, 30.89398792381055],
 [20, 32.028842389602254],
 [21, 33.05718827831063],
 [22, 34.14939231611082],
 [23, 35.19984032732898],
 [24, 36.19142580332634],
 [25, 37.25497055565273],
 [26, 38.20280825884821],
 [27, 39.0680269831439],
 [28, 40.023063095452095],
 [29, 40.829512592167],
 [30, 41.73373958718044],
 [31, 42.55769905296387],
 [32, 43.364264792032806],
 [33, 44.25215131097484],
 [34, 45.14295418727243],
 [35, 46.089254205387604],
 [36, 47.01987534185588],
 [37, 48.132212457011846],
 [38, 49.055432538937886],
 [39, 5

In [64]:
mden

89.40086206896552